<center><img src="../../logo.png" alt="Header" style="width: 800px;"/></center>

@Copyright (C): 2010-2022, Shenzhen Yahboom Tech  
@Author: Liusen  
@Date: 2020-03-10 12:11:02  
@LastEditors: Liusen  
@LastEditTime: 2020-02-18 12:12:02   

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import numpy
import operator
import os
import matplotlib.pyplot as plt


In [ ]:
# Convert image data to (1,1024) vector
def img2vector(filename):
    returnVect = numpy.zeros((1, 1024))
    file = open(filename)
    for i in range(32):
        lineStr = file.readline()
        for j in range(32):
            returnVect[0, 32 * i + j] = int(lineStr[j])
    return returnVect

In [ ]:
# KNN Classifier
def classifier(inX, dataSet, labels, k):
    #numpy shape [0] return the number of rows in the array, shape [1] returns the number of columns
    #MDS
    dataSetSize = dataSet.shape[0]
    #The inverse matrix
    diffMat = numpy.tile(inX, (dataSetSize, 1)) - dataSet
    #subtract a 2-d feature and you multiply it
    sqDiffMat = diffMat ** 2
    #Calculation of distance
    sqDistances = sqDiffMat.sum(axis=1)
    distances = sqDistances ** 0.5
    print ("distances:",distances)
    #Returns the index of the elements in distance sorted from smallest to largest
    sortedDistIndicies = distances.argsort()
    print ("sortDistance:",sortedDistIndicies)
    classCount = {}
    for i in range(k):
        #Take the categories of the first k elements
        voteIlabel = labels[sortedDistIndicies[i]]
        classCount[voteIlabel] = classCount.get(voteIlabel, 0) + 1
    #reverse
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [ ]:
# Test the handwritten number recognition code
def handWritingClassTest(k):
    #Training
    hwLabels = []
    trainingFileList = os.listdir('knn-digits/trainingDigits')
    m = len(trainingFileList)
    trainingMat = numpy.zeros((m, 1024))
    for i in range(m):
        fileNameStr = trainingFileList[i]
        fileStr = fileNameStr.split('.')[0]  
        try:
            classNumStr = int(fileStr.split('_')[0])
        except Exception as e:
            print('Error:', e)
 
        hwLabels.append(classNumStr)
        trainingMat[i, :] = img2vector("knn-digits/trainingDigits/%s" % fileNameStr)
    
    #Test data classification results
    testFileList = os.listdir('knn-digits/testDigits')
    errorCount = 0.0
    mTest = len(testFileList)
    for i in range(mTest):
        fileNameStr = testFileList[i]
        fileStr = fileNameStr.split('.')[0]
        try:
            classNumStr = int(fileStr.split('_')[0])
        except ValueError:
            #pass
            continue
        vectorTest = img2vector("knn-digits/testDigits/%s" % fileNameStr)
        result = classifier(vectorTest, trainingMat, hwLabels, k)
        print("The classification result is:%d,  The real result is:%d" % (result, classNumStr))
        if result != classNumStr:
            errorCount += 1.0
    print("Total number of errors:%d" % errorCount)
    print("Error rate:%f" % (errorCount / mTest))
    return errorCount

In [ ]:
# Choose a different k value to view the classification effect
def selectK():
    x = list()
    y = list()
    for i in range(1, 5):
        x.append(int(i))
        y.append(int(handWritingClassTest(i)))
    plt.plot(x, y)
    # Due to the relatively long execution time of the program, here is a voice reminder when the program execution is completed (for Mac, Windows users remove this line of code)
    # os.system("say 'The program is finished'")
    plt.show()

In [ ]:
# Start the test, a line chart will be generated
selectK()

In [ ]:
# The experimental results prove that taking k = 3 with a better effect

In [ ]:
handWritingClassTest(3)